# 生成电视剧剧本

在这个项目中，你将使用 RNN 创作你自己的[《辛普森一家》](https://zh.wikipedia.org/wiki/%E8%BE%9B%E6%99%AE%E6%A3%AE%E4%B8%80%E5%AE%B6)电视剧剧本。你将会用到《辛普森一家》第 27 季中部分剧本的[数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)。你创建的神经网络将为一个在 [Moe 酒馆](https://simpsonswiki.com/wiki/Moe's_Tavern)中的场景生成一集新的剧本。

## 获取数据
我们早已为你提供了数据`./data/Seinfeld_Scripts.txt`。我们建议你打开文档来看看这个文档内容。

>* 第一步，我们来读入文档，并看几段例子。
* 然后，你需要定义并训练一个 RNN 网络来生成新的剧本！

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)
len(text)

3471464

## 探索数据
使用 `view_line_range` 来查阅数据的不同部分，这个部分会让你对整体数据有个基础的了解。你会发现，文档中全是小写字母，并且所有的对话都是使用 `\n` 来分割的。

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## 实现预处理函数
对数据集进行的第一个操作是预处理。请实现下面两个预处理函数：

- 查询表
- 标记符号

### 查询表
要创建词嵌入，你首先要将词语转换为 id。请在这个函数中创建两个字典：

- 将词语转换为 id 的字典，我们称它为 `vocab_to_int`
- 将 id 转换为词语的字典，我们称它为 `int_to_vocab`

请在下面的元组中返回这些字典
 `(vocab_to_int, int_to_vocab)`

In [53]:
import problem_unittests as tests
from collections import Counter
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get,reverse = True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 0)}
    int_to_vocab = {int_1 : vocab for vocab, int_1 in vocab_to_int.items()}
    print(vocab_to_int)
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'moe_szyslak': 0, 'mike': 1, 'rotch': 2, 'you': 3, 'your': 4, 'to': 5, 'drink': 6, 'the': 7, 'yeah': 8, 'name': 9, 'on': 10, 'hey': 11, 'one': 12, "i'm": 13, 'gonna': 14, 'my': 15, 'homer': 16, 'not': 17, 'problems': 18, 'should': 19, "moe's": 20, 'tavern': 21, 'where': 22, 'elite': 23, 'meet': 24, 'bart_simpson': 25, 'eh': 26, 'hello': 27, 'is': 28, 'there': 29, 'last': 30, 'hold': 31, "i'll": 32, 'check': 33, 'has': 34, 'anybody': 35, 'seen': 36, 'lately': 37, 'listen': 38, 'little': 39, 'puke': 40, 'of': 41, 'these': 42, 'days': 43, 'catch': 44, 'and': 45, 'carve': 46, 'back': 47, 'with': 48, 'an': 49, 'ice': 50, 'pick': 51, 'whats': 52, 'matter': 53, "you're": 54, 'normal': 55, 'effervescent': 56, 'self': 57, 'homer_simpson': 58, 'i': 59, 'got': 60, 'moe': 61, 'give': 62, 'me': 63, 'another': 64, 'forget': 65, 'barney_gumble': 66, 'only': 67, 'enhance': 68, 'social': 69, 'skills': 70}
Tests Passed


### 标记符号的字符串
我们会使用空格当作分隔符，来将剧本分割为词语数组。然而，句号和感叹号等符号使得神经网络难以分辨“再见”和“再见！”之间的区别。

实现函数 `token_lookup` 来返回一个字典，这个字典用于将 “!” 等符号标记为 “||Exclamation_Mark||” 形式。为下列符号创建一个字典，其中符号为标志，值为标记。

- period ( . )
- comma ( , )
- quotation mark ( " )
- semicolon ( ; )
- exclamation mark ( ! )
- question mark ( ? )
- left parenthesis ( ( )
- right parenthesis ( ) )
- dash ( -- )
- return ( \n )

这个字典将用于标记符号并在其周围添加分隔符（空格）。这能将符号视作单独词汇分割开来，并使神经网络更轻松地预测下一个词汇。请确保你并没有使用容易与词汇混淆的标记。与其使用 “dash” 这样的标记，试试使用“||dash||”。

In [41]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    list_1 = [',', '.', '\"', ';', '!','?', '(', ')', '-', '\n' ]
    list_2 = ['||period||', '||comma||', '||quotationmark||', '||semicolon||', 
              '||exclamationmark||', '||questionmark||', '||leftparenthesis||', 
              '||rightparenthesis||', '||dash||', '||return||']
    dict_1 = dict(zip(list_1, list_2))
    return dict_1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## 预处理并保存所有数据
运行以下代码将预处理所有数据，并将它们保存至文件。建议你查看`helpers.py` 文件中的 `preprocess_and_save_data` 代码来看这一步在做什么，但是你不需要修改`helpers.py`中的函数。

In [54]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

NameError: name 'data_dir' is not defined

# 检查点
这是你遇到的第一个检点。如果你想要回到这个 notebook，或需要重新打开 notebook，你都可以从这里开始。预处理的数据都已经保存完毕。

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## 创建神经网络
在本节中，你会构建 RNN 中的必要 Module，以及 前向、后向函数。

### 检查 GPU 访问权限

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## 输入
让我们开始预处理输入数据。我们会使用 [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) 来为数据库提供一个数据格式；以及一个 [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), 该对象会实现 batching，shuffling 以及其他数据迭代功能。

你可以通过传入 特征 和目标 tensors 来创建 TensorDataset，随后创建一个 DataLoader 。
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
 通过 `TensorDataset` 和 `DataLoader` 类来实现  `batch_data` 函数来将 `words` 数据分成 `batch_size` 批次。

>你可以使用 DataLoader 来分批 单词, 但是你可以自由设置 `feature_tensors` 和 `target_tensors` 的大小以及 `sequence_length`。

比如，我们有如下输入:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

你的第一个 `feature_tensor` 会包含:
```
[1, 2, 3, 4]
```
随后的 `target_tensor` 会是接下去的一个字符值:
```
5
```
那么，第二组的`feature_tensor`, `target_tensor` 则如下所示:
```
[2, 3, 4, 5]  # features
6             # target
```

In [3]:
from torch.utils.data import TensorDataset, DataLoader
import torch

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    feature=[]
    target =[]
    for i,word in enumerate(words):
        if i+sequence_length == len(words):
            break
        list_1 = words[i:i+sequence_length]
        list_2 = words[i+sequence_length]
        feature.append(list_1)
        target.append(list_2)
        
    data = TensorDataset(torch.Tensor(feature), torch.Tensor(target))
    data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size,
                                             shuffle =True)
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### 测试你的 dataloader 

你需要改写下述代码来测试 batching 函数，改写后的代码会现在的比较类似。

下面，我们生成了一些测试文本数据，并使用了一个你上面写 dataloader 。然后，我们会得到一些使用`sample_x`输入以及`sample_y`目标生成的文本。

你的代码会返回如下结果(通常是不同的顺序，如果你 shuffle 了你的数据):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### 大小
你的 sample_x 应该是 `(batch_size, sequence_length)`的 大小 或者是(10, 5)， sample_y 应该是 一维的: batch_size (10)。

### 值

你应该也会发现 sample_y, 是 test_text 数据中的*下一个*值。因此，对于一个输入的序列 `[ 28,  29,  30,  31,  32]` ，它的结尾是 `32`, 那么其相应的输出应该是 `33`。

In [4]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 44.,  45.,  46.,  47.,  48.],
        [ 21.,  22.,  23.,  24.,  25.],
        [  3.,   4.,   5.,   6.,   7.],
        [ 22.,  23.,  24.,  25.,  26.],
        [ 41.,  42.,  43.,  44.,  45.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 28.,  29.,  30.,  31.,  32.],
        [ 17.,  18.,  19.,  20.,  21.],
        [ 12.,  13.,  14.,  15.,  16.],
        [ 33.,  34.,  35.,  36.,  37.]])

torch.Size([10])
tensor([ 49.,  26.,   8.,  27.,  46.,  10.,  33.,  22.,  17.,  38.])


---
## 构建神经网络
使用 PyTorch [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module) 来实现一个 循环神经网络 RNN。你需要选择一个 GRU 或者 一个 LSTM。为了完成循环神经网络。为了实现 RNN，你需要实现以下类:
 - `__init__` - 初始化函数
 - `init_hidden` - LSTM/GRU 隐藏组昂泰的初始化函数
 - `forward` - 前向传播函数
 
初始化函数需要创建神经网络的层数，并保存到类。前向传播函数会使用这些网络来进行前向传播，并生成输出和隐藏状态。

在该流程完成后，**该模型的输出是 *最后的* 文字分数结果** 对于每段输入的文字序列，我们只需要输出一个单词，也就是，下一个单词。 

### 提示

1. 确保 lstm 的输出会链接一个 全链接层，你可以参考如下代码 `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. 你可以通过 reshape 模型最后输出的全链接层，来得到最终的文字分数:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
import torch.nn as nn
import numpy as np
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                           dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
        # set class variables
        self.sigmoid = nn.Sigmoid()
        # define model layers
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.08
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.lstm.weight_ih_l0.data.uniform_(-initrange, initrange)
        self.lstm.weight_hh_l0.data.uniform_(-initrange, initrange)
        
        self.lstm.bias_ih_l0.data.zero_()
        self.lstm.bias_hh_l0.data.zero_()
        
        # self.fc.bias.data.zero_()
        self.fc.bias.data.fill_(0)
        # self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.normal_(0.0, (1.0 / np.sqrt(self.fc.in_features)))
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        batch_size = nn_input.size(0)
        
        embeds = self.embedding(nn_input.to(torch.int64))
        
        r_output, hidden = self.lstm(embeds, hidden)
        
        r_output = r_output.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(r_output)
        
        out = self.fc(out)
        
        #sigout = self.sigmoid(out)
        
        sigout = out.view(batch_size, -1, self.output_size)
       
        sigout = sigout[:,-1]


        # return one batch of output word scores and the hidden state
        return sigout, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden
        # initialize hidden state with zero weights, and move to GPU if available


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### 定义前向及后向传播

通过你实现的 RNN 类来进行前向及后项传播。你可以在训练循环中，不断地调用如下代码来实现：
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

函数中需要返回一个批次以及其隐藏状态的loss均值，你可以调用一个函数`RNN(inp, hidden)`来实现。记得，你可以通过调用`loss.item()` 来计算得到该loss。

**如果使用 GPU，你需要将你的数据存到 GPU 的设备上。**

In [7]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    if (train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
  
    h = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output, h = rnn(inp, h)
    loss = criterion(output, target.to(torch.int64))
    loss.backward()
    #nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    
    # move data to GPU, if available
    
    # perform backpropagation and optimization

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), h

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## 神经网络训练

神经网络结构完成以及数据准备完后，我们可以开始训练网络了。

### 训练循环

训练循环是通过 `train_decoder` 函数实现的。该函数将进行 epochs 次数的训练。模型的训练成果会在一定批次的训练后，被打印出来。这个“一定批次”可以通过`show_every_n_batches` 来设置。你会在下一节设置这个参数。

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### 超参数

设置并训练以下超参数:
-  `sequence_length`，序列长度 
-  `batch_size`，分批大小
-  `num_epochs`，循环次数
-  `learning_rate`，Adam优化器的学习率
-  `vocab_size`，唯一标示词汇的数量
-  `output_size`，模型输出的大小 
-  `embedding_dim`，词嵌入的维度，小于 vocab_size
-  `hidden_dim`， 隐藏层维度
-  `n_layers`， RNN的层数
-  `show_every_n_batches`，打印结果的频次

如果模型没有获得你预期的结果，调整 `RNN`类中的上述参数。

In [26]:
import numpy as np
# Data params
# Sequence Length
sequence_length = 6 # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [29]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.002
# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimensi
embedding_dim = 300
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### 训练
下一节，通过预处理数据来训练神经网络。如果你的loss结果不好，可以通过调整超参数来修正。通常情况下，大的隐藏层及层数会带来比较好的效果，但同时也会消耗较长的时间来训练。
> **你应该努力得到一个低于3.5的loss** 

你也可以试试不同的序列长度，该参数表明模型学习的范围大小。

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.546882827758789

Epoch:    1/10    Loss: 5.026365770816803

Epoch:    1/10    Loss: 4.887311793327331

Epoch:    1/10    Loss: 4.844449583530426

Epoch:    1/10    Loss: 4.750012926578521

Epoch:    1/10    Loss: 4.704586074829102

Epoch:    1/10    Loss: 4.642463809967041

Epoch:    1/10    Loss: 4.580439461708069

Epoch:    1/10    Loss: 4.524099811553955

Epoch:    1/10    Loss: 4.567301679611206

Epoch:    1/10    Loss: 4.514655227184296

Epoch:    1/10    Loss: 4.5148422832489015

Epoch:    1/10    Loss: 4.446859915733337

Epoch:    1/10    Loss: 4.512859630107879

Epoch:    1/10    Loss: 4.449492298603058

Epoch:    1/10    Loss: 4.422869925498962

Epoch:    1/10    Loss: 4.4292585582733155

Epoch:    1/10    Loss: 4.424505758285522

Epoch:    1/10    Loss: 4.414570638179779

Epoch:    1/10    Loss: 4.4098689012527466

Epoch:    1/10    Loss: 4.405800260543823

Epoch:    1/10    Loss: 4.379049689292907

Epoch:    1/10    Loss:

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### 问题: 你如何决定你的模型超参数？
比如，你是否试过不同的 different sequence_lengths 并发现哪个使得模型的收敛速度变化？那你的隐藏层数和层数呢？你是如何决定使用这个网络参数的？

**答案:** 调参失败，损失变化基本没有。

---
# 检查点

通过运行上面的训练单元，你的模型已经以`trained_rnn`名字存储，如果你存储了你的notebook， **你可以在之后的任何时间来访问你的代码和结果**. 下述代码可以帮助你重载你的结果!

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## 生成电视剧剧本
你现在可以生成你的“假”电视剧剧本啦！

### 生成文字
你的神经网络会不断重复生成一个单词，直到生成满足你要求长度的剧本。使用 `generate` 函数来完成上述操作。首先，使用 `prime_id` 来生成word id，之后确定生成文本长度 `predict_len`。同时， topk 采样来引入文字选择的随机性!

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### 生成一个新剧本
是时候生成一个剧本啦。设置`gen_length` 剧本长度，设置 `prime_word`为以下任意词来开始生成吧:
- "jerry"
- "elaine"
- "george"
- "kramer"

你可以把prime word 设置成 _任意 _ 单词, 但是使用名字开始会比较好(任何其他名字也是可以哒!)

In [33]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = "kramer" # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


kramer:(to elaine) you don't have to be a little.

elaine: oh, no!

kramer:(to kramer) i can't.

kramer:(to jerry) hey, jerry, it's a big idea.

george: oh, no.

jerry: what?

jerry:(to jerry) i don't know, but it's like that...

jerry: i don't know.

jerry:(pointing) well, i just got to see you.

jerry: oh, no, no.

jerry:(to jerry) what is the point of the guy...

george: oh, you have to go to the bathroom, he doesn't think i should get it...

jerry: what is it?

george: i can't do it.

george:(to jerry) what is this?

jerry:(to george) i know.

george: well, what are you going to do?

jerry: yeah, yeah, yeah.

jerry: oh, no... i don't think you can be a good time.

george: well what are they doing here?!

newman:(looking) i think i could be a little..

jerry: i don't know...(jerry enters.)

elaine: what are you doing?

jerry: i don't know.

jerry: oh yeah, yeah, yeah, i was gonna get out of the building. you have to go?

newman: i think i was a little bit.(to the other man, and then

#### 存下你最爱的片段

一旦你发现一段有趣或者好玩的片段，就把它存下啦！

In [34]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# 这个电视剧剧本是无意义的
如果你的电视剧剧本不是很有逻辑也是ok的。下面是一个例子。

### 生成剧本案例

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.


如果这个电视剧剧本毫无意义，那也没有关系。我们的训练文本不到一兆字节。为了获得更好的结果，你需要使用更小的词汇范围或是更多数据。幸运的是，我们的确拥有更多数据！在本项目开始之初我们也曾提过，这是[另一个数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)的子集。我们并没有让你基于所有数据进行训练，因为这将耗费大量时间。然而，你可以随意使用这些数据训练你的神经网络。当然，是在完成本项目之后。
# 提交项目
在提交项目时，请确保你在保存 notebook 前运行了所有的单元格代码。请将 notebook 文件保存为 "dlnd_tv_script_generation.ipynb"，并将它作为 HTML 文件保存在 "File" -> "Download as" 中。请将 "helper.py" 和 "problem_unittests.py" 文件一并打包成 zip 文件提交。